In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [2]:
output_dir = "graficos_adosamiento_edificio"
os.makedirs(output_dir, exist_ok=True)

In [3]:
nepal = pd.read_csv("eq2015.csv")
fuente_1 = "https://www.kaggle.com/datasets/sanskarnegi/nepal-earthquake-2015"

In [4]:
distritos = nepal["district_name"].unique()
distritos

array(['OKHALDHUNGA', 'SINDHULI', 'RAMECHHAP', 'DOLAKHA', 'SINDHUPALCHOK',
       'KAVRE', 'NUWAKOT', 'RASUWA', 'DHADING', 'MAKWANPUR', 'GORKHA'],
      dtype=object)

In [5]:
graficos_por_distrito = {}

In [6]:
for distrito in distritos:
    df_distrito = nepal[nepal["district_name"] == distrito]
    df_grouped = df_distrito.groupby(["position", "damage_grade"]).size().unstack()

    # Crear la figura
    fig, ax = plt.subplots(figsize=(6, 4))
    df_grouped.plot(kind='bar', stacked=True, colormap='viridis', ax=ax)

    # Etiquetas
    plt.xlabel('Tipo de adosamiento')
    plt.ylabel('Número de edificios')
    plt.title(f'Distrito: {distrito}')

    # Guardar la imagen en memoria
    img_path = os.path.join(output_dir, f"{distrito}.png")
    plt.savefig(img_path, bbox_inches="tight", dpi=150)
    plt.close(fig)

    # Guardar ruta de la imagen
    graficos_por_distrito[distrito] = img_path